# 💰 Dự đoán giá Bitcoin với LSTM + Google Sheets + Gradio UI

In [ ]:
# 📦 Cài đặt thư viện cần thiết
!pip install yfinance gspread oauth2client gradio tensorflow

In [ ]:
# 🔐 Định nghĩa hàm log_prediction để ghi vào Google Sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

def log_prediction(actual_price, predicted_price):
    scope = ["https://spreadsheets.google.com/feeds",
             "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("btc-predictor-465418-724dc76c0ac9.json", scope)
    client = gspread.authorize(creds)
    sheet = client.open_by_key("1DyunpedkCQ0NZhUf2Ecr3bVbigbK3-AXrYPwQv8xgfE").worksheet("BTC_Log")
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    sheet.append_row([now, actual_price, predicted_price])
    print("✅ Đã ghi vào Google Sheet:", [now, actual_price, predicted_price])

In [ ]:
# 📊 Lấy dữ liệu thực tế từ CoinGecko
import requests
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def fetch_recent_prices(days=30):
    url = f"https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days={days}&interval=daily"
    data = requests.get(url).json()
    prices = np.array(data['prices'])[:, 1].reshape(-1, 1)
    return prices

recent_prices = fetch_recent_prices(30)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(recent_prices)
X_real = np.array([scaled])

In [ ]:
# 🤖 Dự đoán bằng mô hình LSTM đã lưu
from tensorflow.keras.models import load_model
model = load_model('btc_lstm_model.h5')

predicted_scaled = model.predict(X_real)
predicted_price = scaler.inverse_transform(predicted_scaled)

actual_price = recent_prices[-1][0]
predicted_value = predicted_price[0][0]

print(f"💰 Dự đoán giá Bitcoin ngày tiếp theo: ${predicted_value:,.2f}")

log_prediction(actual_price, predicted_value)

In [ ]:
# 🖼️ Hàm show_predictions để hiển thị bảng + biểu đồ từ Google Sheet
import pandas as pd
import matplotlib.pyplot as plt
import io
from PIL import Image

def show_predictions():
    creds = ServiceAccountCredentials.from_json_keyfile_name("btc-predictor-465418-724dc76c0ac9.json", [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive"]
    )
    client = gspread.authorize(creds)
    sheet = client.open_by_key("1DyunpedkCQ0NZhUf2Ecr3bVbigbK3-AXrYPwQv8xgfE").worksheet("BTC_Log")
    df = pd.DataFrame(sheet.get_all_records())

    if df.empty:
        return pd.DataFrame(), None

    df['Time'] = pd.to_datetime(df['Time'])
    df = df.sort_values('Time').tail(5)

    plt.figure(figsize=(10, 4))
    plt.plot(df['Time'], df['Actual Price'], label='Giá thật', marker='o')
    plt.plot(df['Time'], df['Predicted Price'], label='Giá dự đoán', marker='x')
    plt.grid(True); plt.legend(); plt.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format='png'); buf.seek(0)
    image = Image.open(buf)
    return df, image

In [ ]:
# 🚀 Tạo giao diện Gradio
import gradio as gr

demo = gr.Interface(
    fn=show_predictions,
    inputs=[],
    outputs=["dataframe", "image"],
    title="💰 Dự đoán giá Bitcoin",
    description="Hiển thị dữ liệu và biểu đồ từ Google Sheet"
)
demo.launch(share=True)